In [1]:
!pip install fastapi uvicorn pyngrok nest_asyncio streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


In [10]:
import pickle
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import nest_asyncio
import uvicorn
from threading import Thread
from pyngrok import ngrok
import os
from google.colab import userdata

# Load the model
model_path = "/content/best_model.pkl"  # Ensure this file exists in the Colab environment
with open(model_path, "rb") as f:
    model = pickle.load(f)

# Initialize FastAPI app
app = FastAPI()

class PredictionInput(BaseModel):
    Education: str
    JoiningYear: int
    City: str
    PaymentTier: int
    Age: int
    Gender: str
    EverBenched: int
    ExperienceInCurrentDomain: int

@app.post("/predict")
def predict(input_data: PredictionInput):
    input_df = pd.DataFrame([input_data.dict()])
    prediction = model.predict(input_df)
    return {"prediction": int(prediction[0])}

# Allow FastAPI to run in Colab
nest_asyncio.apply()

# Start FastAPI server on port 8000
def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000)

fastapi_thread = Thread(target=run_fastapi, daemon=True)
fastapi_thread.start()

# Expose FastAPI through ngrok

auth_token=userdata.get('Ngrok')
ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(8000)
fastapi_url = ngrok_tunnel.public_url
print(f"FastAPI public URL: {fastapi_url}")

# Save FastAPI URL to a file for Streamlit to use
with open("fastapi_url.txt", "w") as f:
    f.write(fastapi_url)


INFO:     Started server process [925]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


FastAPI public URL: https://27bf-34-168-166-119.ngrok-free.app


In [11]:
%%writefile app.py
import streamlit as st
import requests

# Load FastAPI public URL dynamically
with open("fastapi_url.txt", "r") as f:
    api_url = f.read().strip() + "/predict"

st.title("Employee Attrition Prediction")
st.write("Use this app to predict whether an employee will leave or stay.")

# Input fields for user data
education = st.selectbox("Education", options=["Bachelors", "Masters", "PHD"])
joining_year = st.number_input("Joining Year", min_value=2000, max_value=2025, step=1, value=2015)
city = st.selectbox("City", options=["Bangalore", "Pune", "New Delhi"])
payment_tier = st.selectbox("Payment Tier", options=[1, 2, 3])
age = st.number_input("Age", min_value=18, max_value=65, step=1, value=30)
gender = st.selectbox("Gender", options=["Male", "Female"])
ever_benched = st.selectbox("Ever Benched", options=[0, 1], format_func=lambda x: "No" if x == 0 else "Yes")
experience_in_current_domain = st.number_input(
    "Experience in Current Domain (years)", min_value=0, max_value=80, step=1, value=5
)

if st.button("Predict"):
    input_data = {
        "Education": education,
        "JoiningYear": joining_year,
        "City": city,
        "PaymentTier": payment_tier,
        "Age": age,
        "Gender": gender,
        "EverBenched": ever_benched,
        "ExperienceInCurrentDomain": experience_in_current_domain,
    }
    try:
        response = requests.post(api_url, json=input_data)
        response.raise_for_status()
        response_data = response.json()
        prediction = "Stay" if response_data["prediction"] == 0 else "Leave"
        st.success(f"Prediction: {prediction}")
    except requests.exceptions.RequestException as e:
        st.error(f"Error connecting to the prediction API: {e}")


Overwriting app.py


In [12]:
# Start Streamlit server
from pyngrok import ngrok
import os

streamlit_thread = Thread(target=lambda: os.system("streamlit run app.py --server.port 8501"), daemon=True)
streamlit_thread.start()

# Expose Streamlit through ngrok
streamlit_url = ngrok.connect(8501)
print(f"Streamlit public URL: {streamlit_url}")


Streamlit public URL: NgrokTunnel: "https://6e0d-34-168-166-119.ngrok-free.app" -> "http://localhost:8501"
